In [1]:
# ─── 기본 설정 ────────────────────────────────
import os
import io
import json
import base64
from datetime import datetime
from dotenv import load_dotenv

# ─── 이미지 및 EXIF 처리 ──────────────────────
from PIL import Image
import exifread

# ─── OpenAI 및 LangChain ─────────────────────
import openai
from openai import OpenAI

# LangChain 관련 모듈
from langchain_openai import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.prompts import PromptTemplate
from langchain.schema import SystemMessage, HumanMessage


In [ ]:
# pip install --upgrade openai langchain langchain-openai exifread python-dotenv

In [2]:
# 환경 변수 로드 (.env에 OPENAI_API_KEY 포함되어 있어야 함)
load_dotenv()
client = OpenAI()
chat_model = ChatOpenAI(temperature=0.9)

# 이미지 압축 함수 (base64 변환 포함)
def compress_image(image_path, size=(512, 512)):
    img = Image.open(image_path)
    img = img.resize(size)
    buffer = io.BytesIO()
    img.save(buffer, format="JPEG")
    return base64.b64encode(buffer.getvalue()).decode("utf-8")

# 이미지 촬영 시간 기준 정렬 (EXIF 사용)
def extract_datetime_from_image(image_path):
    with open(image_path, 'rb') as f:
        tags = exifread.process_file(f, stop_tag="EXIF DateTimeOriginal")
    datetime_str = tags.get("EXIF DateTimeOriginal")
    if datetime_str:
        return datetime.strptime(str(datetime_str), "%Y:%m:%d %H:%M:%S")
    return None

def sort_all_images(image_folder):
    images = []
    for filename in os.listdir(image_folder):
        if filename.lower().endswith(('.jpg', '.jpeg')):
            path = os.path.join(image_folder, filename)
            dt = extract_datetime_from_image(path)
            if dt:
                images.append((filename, dt))
    return sorted(images, key=lambda x: x[1])

# 캡션 캐시 로딩 및 저장
def load_cached_captions(cache_path):
    if os.path.exists(cache_path):
        with open(cache_path, 'r', encoding='utf-8') as f:
            return json.load(f)
    return {}

def save_cached_captions(cache_path, data):
    with open(cache_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

# 이미지 → 캡션 생성 (OpenAI GPT-4 Vision 사용)
def image_to_caption(image_path, prompt="이 이미지를 사실적으로 묘사해줘."):
    b64_img = compress_image(image_path)
    response = client.chat.completions.create(
        model="gpt-4-turbo-2024-04-09",
        messages=[
            {"role": "system", "content": "너는 사실적인 이미지 설명을 잘 하는 작가야."},
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{b64_img}"}},
                ],
            },
        ],
        max_tokens=300
    )
    return response.choices[0].message.content

# LangChain 다이어리 프롬프트
diary_prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 감성적인 일기 작가야."),
    ("human", 
    """아래 사진 설명들을 바탕으로 하루를 감성적으로 정리해줘.

- 사진 설명들: {image_caption}
- 감정: {emotion}
- 키워드: {keywords}
- 장소: {place}

조건: 너무 형식적이지 않고, 자연스럽고 따뜻한 300자로 써줘.""")
])

# 전체 파이프라인 실행
folder_path = r"C:\Users\Admin\Desktop\프로젝트\일기생성\test"  # 이미지 폴더 경로
cache_path = os.path.join(folder_path, "caption_cache.json")

sorted_images = sort_all_images(folder_path)
caption_cache = load_cached_captions(cache_path)

all_captions = []
for filename, dt in sorted_images:
    if filename in caption_cache:
        caption = caption_cache[filename]
    else:
        image_path = os.path.join(folder_path, filename)
        caption = image_to_caption(image_path)
        caption_cache[filename] = caption
    all_captions.append(caption)

# 캐시 저장
save_cached_captions(cache_path, caption_cache)

# 전체 캡션을 하나의 문장으로 합침
combined_caption = " ".join(all_captions)

# LangChain 프롬프트 구성
messages = diary_prompt.format_messages(
    image_caption=combined_caption,
    emotion="아주 행복했던 하루였어",  # 추후 자동 감정 추론 가능
    keywords="햇살, 조용한 거리, 여행",
    place=" "
)

# 일기 생성
diary = chat_model.invoke(messages).content

# 결과 출력
print("전체 이미지 기반 하루 요약 일기")
print(diary)


FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: 'C:\\Users\\Admin\\Desktop\\프로젝트\\일기생성\\test'

In [ ]:


# 환경 변수 로드 (.env에 OPENAI_API_KEY 포함되어 있어야 함)
load_dotenv()
client = OpenAI()
chat_model = ChatOpenAI(temperature=0.9)

# 이미지 압축 함수 (base64 변환 포함)
def compress_image(image_path, size=(512, 512)):
    img = Image.open(image_path)
    img = img.resize(size)
    buffer = io.BytesIO()
    img.save(buffer, format="JPEG")
    return base64.b64encode(buffer.getvalue()).decode("utf-8")

# 이미지 촬영 시간 기준 정렬 (EXIF 사용)
def extract_datetime_from_image(image_path):
    with open(image_path, 'rb') as f:
        tags = exifread.process_file(f, stop_tag="EXIF DateTimeOriginal")
    datetime_str = tags.get("EXIF DateTimeOriginal")
    if datetime_str:
        return datetime.strptime(str(datetime_str), "%Y:%m:%d %H:%M:%S")
    return None

def sort_all_images(image_folder):
    images = []
    for filename in os.listdir(image_folder):
        if filename.lower().endswith(('.jpg', '.jpeg')):
            path = os.path.join(image_folder, filename)
            dt = extract_datetime_from_image(path)
            if dt:
                images.append((filename, dt))
    return sorted(images, key=lambda x: x[1])

# 캡션 캐시 로딩 및 저장
def load_cached_captions(cache_path):
    if os.path.exists(cache_path):
        with open(cache_path, 'r', encoding='utf-8') as f:
            return json.load(f)
    return {}

def save_cached_captions(cache_path, data):
    with open(cache_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

# 이미지 → 캡션 생성 (OpenAI GPT-4 Vision 사용)
def image_to_caption(image_path, prompt="이 이미지를 사실적으로 묘사해줘."):
    b64_img = compress_image(image_path)
    response = client.chat.completions.create(
        model="gpt-4-turbo-2024-04-09",
        messages=[
            {"role": "system", "content": "너는 사실적인 이미지 설명을 잘 하는 작가야."},
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{b64_img}"}},
                ],
            },
        ],
        max_tokens=300
    )
    return response.choices[0].message.content

# LangChain 다이어리 프롬프트
diary_prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 섬세한 감성과 관찰력을 가진 일기 작가야. 독자가 하루의 분위기를 생생히 떠올릴 수 있도록, 자연스럽고 따뜻한 언어로 장면과 감정을 표현해줘."),
    ("human", 
    """아래 사진 설명들을 바탕으로 하루를 감성적으로 정리해줘.

- 사진 설명들: {image_caption}
- 감정: {emotion}
- 키워드: {keywords}
- 장소: {place}

조건: 
- 너무 형식적이지 않고 자연스럽고 따뜻한 문체로 작성해줘.  
- 감정과 분위기를 중심으로, 하루를 회상하듯 써줘.  
- 분량은 300자 정도로 구성해줘.""")
])

# 전체 파이프라인 실행
folder_path = r"C:\Users\Admin\Desktop\프로젝트\일기생성\test"  # 이미지 폴더 경로
cache_path = os.path.join(folder_path, "caption_cache.json")

sorted_images = sort_all_images(folder_path)
caption_cache = load_cached_captions(cache_path)

all_captions = []
for filename, dt in sorted_images:
    if filename in caption_cache:
        caption = caption_cache[filename]
    else:
        image_path = os.path.join(folder_path, filename)
        caption = image_to_caption(image_path)
        caption_cache[filename] = caption
    all_captions.append(caption)

# 캐시 저장
save_cached_captions(cache_path, caption_cache)

# 전체 캡션을 하나의 문장으로 합침
combined_caption = " ".join(all_captions)

# LangChain 프롬프트 구성
messages = diary_prompt.format_messages(
    image_caption=combined_caption,
    emotion="아주 행복했던 하루였어",  # 추후 자동 감정 추론 가능
    keywords="햇살, 조용한 거리, 여행",
    place=" "
)

# 일기 생성
diary = chat_model.invoke(messages).content

# 결과 출력
print("전체 이미지 기반 하루 요약 일기")
print(diary)


전체 이미지 기반 하루 요약 일기
오늘은 햇살 가득한 아름다운 날이었어. 작은 조용한 골목길을 걷다가 발견한 동상이 마음을 끌었어. 검은 동상이 큰 바위 위에 앉아, 머리카락이 바람에 흩날리며 무엇을 보고 있는지 궁금했어. 주변에는 푸른 하늘과 건물들이 조화를 이루며 평화로운 분위기를 연출했어. 그 곳에서 아침을 즐기는 모습도 보였고, 동물들이 자유롭게 뛰노는 모습도 눈에 들어왔어. 마지막으로는 동물 모양 조각들이 담긴 접시가 마음을 설레게 했어. 모든 순간이 행복하고 편안한 기분으로 가득 찼던 하루였어. 오늘은 정말 특별한 여행이었어.


In [6]:
# 환경 변수 로드 (.env에 OPENAI_API_KEY 포함되어 있어야 함)
load_dotenv()
client = OpenAI()
chat_model = ChatOpenAI(temperature=0.9)

# 이미지 압축 함수 (base64 변환 포함)
def compress_image(image_path, size=(512, 512)):
    img = Image.open(image_path)
    img = img.resize(size)
    buffer = io.BytesIO()
    img.save(buffer, format="JPEG")
    return base64.b64encode(buffer.getvalue()).decode("utf-8")

# 이미지 촬영 시간 기준 정렬 (EXIF 사용)
def extract_datetime_from_image(image_path):
    with open(image_path, 'rb') as f:
        tags = exifread.process_file(f, stop_tag="EXIF DateTimeOriginal")
    datetime_str = tags.get("EXIF DateTimeOriginal")
    if datetime_str:
        return datetime.strptime(str(datetime_str), "%Y:%m:%d %H:%M:%S")
    return None


def sort_all_images(image_folder):
    images = []
    for filename in os.listdir(image_folder):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            path = os.path.join(image_folder, filename)
            dt = extract_datetime_from_image(path)
            images.append((filename, dt if dt else filename))
    return sorted(images, key=lambda x: x[1])

# 캡션 캐시 로딩 및 저장
def load_cached_captions(cache_path):
    if os.path.exists(cache_path):
        with open(cache_path, 'r', encoding='utf-8') as f:
            return json.load(f)
    return {}

def save_cached_captions(cache_path, data):
    with open(cache_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

# 이미지 → 캡션 생성 (OpenAI GPT-4 Vision 사용)
def image_to_caption(image_path, prompt="이 이미지를 사실적으로 묘사해줘."):
    b64_img = compress_image(image_path)
    response = client.chat.completions.create(
        model="gpt-4-turbo-2024-04-09",
        messages=[
            {"role": "system", "content": "너는 사실적인 이미지 설명을 잘 하는 작가야."},
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{b64_img}"}},
                ],
            },
        ],
        max_tokens=300
    )
    return response.choices[0].message.content

# LangChain 다이어리 프롬프트
diary_prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 사실적이고 담백한 어조로 하루를 정리하는 일기 작가야. 감정에 치우치지 않고, 사진에 담긴 상황과 분위기를 자연스럽게 설명해줘"),
    ("human", 
    """다음은 오늘 하루 동안 찍힌 사진 설명이야. 이 설명들을 참고해서, 하루를 돌아보는 일기 한 단락을 작성해줘.

- 사진 설명들: {image_caption}
- 감정: {emotion}
- 키워드: {keywords}
- 장소: {place}

조건:
- 너무 감성적이거나 시적인 표현은 피하고, 담백하고 솔직한 문체로 써줘.
- 일상의 흐름을 회고하듯 간결하게 정리해줘.
- 300~500자 정도로 작성해줘.""")
])

# 전체 파이프라인 실행
folder_path = r"C:\Users\Admin\Desktop\프로젝트\일기생성\test2"  # 이미지 폴더 경로
cache_path = os.path.join(folder_path, "caption_cache.json")

sorted_images = sort_all_images(folder_path)
caption_cache = load_cached_captions(cache_path)

all_captions = []
for filename, dt in sorted_images:
    if filename in caption_cache:
        caption = caption_cache[filename]
    else:
        image_path = os.path.join(folder_path, filename)
        caption = image_to_caption(image_path)
        caption_cache[filename] = caption
    all_captions.append(caption)

# 캐시 저장
save_cached_captions(cache_path, caption_cache)

# 전체 캡션을 하나의 문장으로 합침
combined_caption = " ".join(all_captions)

# LangChain 프롬프트 구성
messages = diary_prompt.format_messages(
    image_caption=combined_caption,
    emotion="아주 행복했던 하루였어",  # 추후 자동 감정 추론 가능
    keywords="햇살, 조용한 거리, 여행",
    place="국내 일본 마을"
)

# 일기 생성
diary = chat_model.invoke(messages).content

# 결과 출력
print("전체 이미지 기반 하루 요약 일기")
print(diary)


전체 이미지 기반 하루 요약 일기
오늘은 일본 마을을 여행하며 소중한 시간을 보냈어. 출입구에서는 "정지"라는 글씨가 적힌 파란 표지판이 반가운 인사를 해주었고, 친구들과 함께 찍은 셀카 속 우리는 행복한 미소를 띄었어. 마을 곳곳에는 전통적인 건축물과 크리스마스 장식이 눈에 띄었는데, 따스한 분위기 속에서 카페에 앉아 다채로운 소품들을 구경하는 즐거운 시간을 보냈어. 나는 테이블 아래에서 주황색 고양이와 놀며 평화로운 순간을 나누었고, 거리에선 축제 분위기 속에서 노란 불빛이 반짝이는 풍경을 즐겼어. 저녁이 되어 즐거움 가득한 상점가에서는 크리스마스 장식과 함께 휴식을 취하는 사람들이 보였어. 마지막으로 오래된 공방 속에서는 목공 작업장의 역사와 전통을 엿볼 수 있는 소중한 경험을 했어. 춥고 함께한 하루였지만, 햇살 가득한 기억들이 마음을 따뜻하게 해주었어.
